<a href="https://colab.research.google.com/github/Gape-42/IniciacaoCientifica/blob/main/Simulated_Annealing_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importando bibliotecas**




In [ ]:
import pandas as pd
import random
import math
import copy
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Carregando conjunto de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_de_dados = 'Vertebral.csv' #Alterar nome para escolher nova base de dados
caminho_dos_dados = '/content/drive/MyDrive/Colab Notebooks/Iniciação Científica/Bases de dados/'

Mounted at /content/drive


In [ ]:
dados = pd.read_csv(caminho_dos_dados + base_de_dados)
dados = shuffle(dados)
X = dados.iloc[:,:-1]
Y = dados.iloc[:,-1]

Gerando os conjuntos de treino, teste e validação

In [ ]:
x_treino,x_temp,y_treino,y_temp=train_test_split(X,Y,test_size=0.5,stratify=Y)
x_validacao,x_teste,y_validacao,y_teste=train_test_split(x_temp,y_temp,test_size=0.5, stratify = y_temp)

 Constantes

In [ ]:
NUMERO_DE_ARVORES_FLORESTA = 200
INTERVALO_FLORESTA = 20
SA_MAX = 5
ALFA = 0.995
TEMPERATURA_MINIMA = 0.005

Definindo os individuos

In [ ]:
class individuos:
  def __init__(self, numero_de_arvores, criterio, profundidade, min_samples_split, min_samples_leaf, fitness):
    self.numero_de_arvores = numero_de_arvores
    self.criterio = criterio
    self.profundidade = profundidade
    self.min_samples_split = min_samples_split
    self.min_samples_leaf = min_samples_leaf
    self.fitness = fitness

Funções

In [ ]:
def random_forest_validacao(individuo):
  RF = RandomForestClassifier(n_estimators=individuo.numero_de_arvores, criterion=individuo.criterio, max_depth=individuo.profundidade, min_samples_split=individuo.min_samples_split, min_samples_leaf=individuo.min_samples_leaf)
  RF.fit(x_treino,y_treino)
  opiniao = RF.predict(x_validacao)
  return accuracy_score(y_validacao, opiniao)

def random_forest_teste(individuo):
  RF = RandomForestClassifier(n_estimators=individuo.numero_de_arvores, criterion=individuo.criterio, max_depth=individuo.profundidade, min_samples_split=individuo.min_samples_split, min_samples_leaf=individuo.min_samples_leaf)
  RF.fit(x_treino,y_treino)
  opiniao = RF.predict(x_teste)
  return accuracy_score(y_teste, opiniao)

def printarIndividuo(individuo):
  print("numero_de_arvores:\t", individuo.numero_de_arvores)
  print("critério:\t\t", individuo.criterio)
  print("profundidade:\t\t", individuo.profundidade)
  print("min_samples_split:\t", individuo.min_samples_split)
  print("min_samples_leaf:\t", individuo.min_samples_leaf)
  print("fitness:\t\t", individuo.fitness)
  print()

def metropolis():
    return random.random()

def gerarIndividuoAleatorio():
    individuo = individuos(None, None, None, None, None, None)
    individuo.numero_de_arvores = random.randint(1, NUMERO_DE_ARVORES_FLORESTA/INTERVALO_FLORESTA) * INTERVALO_FLORESTA
    individuo.criterio = random.choice(["gini", "entropy"])
    individuo.profundidade = random.randint(1, 10)                  #
    individuo.min_samples_split = random.randint(2, 15)             #É preciso alterar?
    individuo.min_samples_leaf = random.randint(1, 10)              #
    individuo.fitness = 0
    return individuo

#Esolhe um atributo para modificar e fazer um vizinho
def escolherAtributo():
    return random.choice(["numero_de_arvores", "criterio", "profundidade", "min_samples_split", "min_samples_leaf"])

def gerarVizinho(individuo, Atributo):
    vizinho = copy.copy(individuo)

    if Atributo == "numero_de_arvores":
        vizinho.numero_de_arvores = random.randint(1, NUMERO_DE_ARVORES_FLORESTA/INTERVALO_FLORESTA) * INTERVALO_FLORESTA
    if Atributo == "criterio":
        vizinho.criterio = random.choice(["gini", "entropy"])
    if Atributo == "profundidade":
        vizinho.profundidade = random.randint(1, 10)
    if Atributo == "min_samples_split":
        vizinho.min_samples_split = random.randint(2, 15)
    if Atributo == "min_samples_leaf":
        vizinho.min_samples_leaf = random.randint(1, 10)

    return vizinho

#Variação da solução atual
#∆E = Ei+1 - Ei
def variacaoSolucao(individuo, vizinho):
    return vizinho.fitness - individuo.fitness

#Probabilidade de aceitar piora (minimização)
# P(∆E) = e^(∆E/T)
def probabilidadePiora(variacao, temperatura):
    return math.exp(variacao / temperatura)

#Resfriamento
def resfriamento(temperatura):
    return temperatura * ALFA

#Gerar temperatura inicial
def temperaturaInicial():
    ind1 = gerarIndividuoAleatorio()
    vizinhos = []
    TempInicial = 0
    for i in range (3):
      vizinhos.append(gerarVizinho(ind1, escolherAtributo()))
      vizinhos[i].fitness = random_forest_validacao(vizinhos[i])

    for i in range (3):
      TempInicial = TempInicial + vizinhos[i].fitness
    TempInicial = TempInicial/3

    return TempInicial

Iniciando o algoritmo

In [ ]:
solucao_atual = gerarIndividuoAleatorio()
solucao_atual.fitness = random_forest_validacao(solucao_atual)
melhor_individuo = solucao_atual
temperatura = temperaturaInicial()
iteracao = 0

#gráfico
temperaturas = []
melhores_individuos = []

while(temperatura > TEMPERATURA_MINIMA):
    #vetores para fazer o gráfico
    temperaturas.append(temperatura)
    melhores_individuos.append(melhor_individuo.fitness)

    while(iteracao < SA_MAX):
        iteracao += 1
        vizinho = gerarVizinho(solucao_atual, escolherAtributo())
        vizinho.fitness = random_forest_validacao(vizinho)
        delta = variacaoSolucao(solucao_atual, vizinho)

        if(delta > 0):
            solucao_atual = vizinho
            if(vizinho.fitness > melhor_individuo.fitness):
                melhor_individuo = vizinho
        else:
            x = metropolis()
            piora = probabilidadePiora(delta, temperatura)
            if(x < piora):
                solucao_atual = vizinho

    temperatura = resfriamento(temperatura)
    iteracao = 0

solucao_atual = melhor_individuo
printarIndividuo(solucao_atual)

numero_de_arvores:	 20
critério:		 entropy
profundidade:		 2
min_samples_split:	 2
min_samples_leaf:	 5
fitness:		 0.8831168831168831



Gerando o gráfico

In [ ]:
quantidade_temperaturas = len(temperaturas)
id_temperaturas =  list(range(quantidade_temperaturas, 0, -1))
# ax1 = plt.gca()
# ax1.invert_xaxis()
plt.grid(True)
plt.plot(id_temperaturas, temperaturas, label="Temperatura")
plt.plot(id_temperaturas, melhores_individuos, label="Fitness")
plt.legend()
plt.show()

NameError: name 'temperaturas' is not defined

Testando a melhor configuração

In [ ]:
media_acuracia = 0
for i in range(100):
    acuracia  = random_forest_teste(solucao_atual)
    # print(acuracia)
    media_acuracia += acuracia

media_acuracia /= 100

printarIndividuo(solucao_atual)
print("Média acurácia: ", media_acuracia)

numero_de_arvores:	 20
critério:		 entropy
profundidade:		 2
min_samples_split:	 2
min_samples_leaf:	 5
fitness:		 0.8831168831168831

Média acurácia:  0.8066666666666664


In [ ]:
# numero_de_arvores:	 20
# critério:		 entropy
# profundidade:		 3
# min_samples_split:	 4
# min_samples_leaf:	 2
# fitness:		 0.8571428571428571

# Média acurácia:  0.7500000000000006

# numero_de_arvores:	 20
# critério:		 entropy
# profundidade:		 5
# min_samples_split:	 10
# min_samples_leaf:	 5
# fitness:		 0.8831168831168831

# Média acurácia:  0.8425641025641022

# numero_de_arvores:	 20
# critério:		 entropy
# profundidade:		 10
# min_samples_split:	 13
# min_samples_leaf:	 10
# fitness:		 0.9090909090909091

# Média acurácia:  0.8542307692307685

# numero_de_arvores:	 80
# critério:		 gini
# profundidade:		 9
# min_samples_split:	 2
# min_samples_leaf:	 1
# fitness:		 0.9090909090909091

# Média acurácia:  0.8114102564102559

# numero_de_arvores:	 80
# critério:		 gini
# profundidade:		 2
# min_samples_split:	 15
# min_samples_leaf:	 1
# fitness:		 0.8961038961038961

# Média acurácia:  0.8326923076923068

# numero_de_arvores:	 20
# critério:		 entropy
# profundidade:		 2
# min_samples_split:	 2
# min_samples_leaf:	 5
# fitness:		 0.8831168831168831

# Média acurácia:  0.8042307692307686